# Import lib

In [ ]:
!pip install pythainlp -qq
!pip install epitran -qq
!pip install sklearn_crfsuite -qq
!pip install tensorflow deepcut -qq
!pip install attacut -qq
!pip install deepcut -qq

     |████████████████████████████████| 11.5 MB 7.7 MB/s 
     |████████████████████████████████| 148 kB 8.0 MB/s 
     |████████████████████████████████| 73 kB 1.5 MB/s 
     |████████████████████████████████| 1.2 MB 46.5 MB/s 
     |████████████████████████████████| 743 kB 8.4 MB/s 
     |████████████████████████████████| 2.0 MB 10.5 MB/s 
     |████████████████████████████████| 462 kB 70.9 MB/s 
     |████████████████████████████████| 1.3 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 37.6 MB/s 
     |████████████████████████████████| 87 kB 5.5 MB/s 
     |████████████████████████████████| 473 kB 54.2 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 


In [ ]:
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import re
import string
import os
from tqdm import tqdm
import glob
import pythainlp
from pythainlp import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

# **Common function**

In [ ]:
#Edit disntance from geek
def editDistance(str1, str2, m, n):

	# If first string is empty, the only option is to
	# insert all characters of second string into first
	if m == 0:
		return n

	# If second string is empty, the only option is to
	# remove all characters of first string
	if n == 0:
		return m

	# If last characters of two strings are same, nothing
	# much to do. Ignore last characters and get count for
	# remaining strings.
	if str1[m-1] == str2[n-1]:
		return editDistance(str1, str2, m-1, n-1)

	# If last characters are not same, consider all three
	# operations on last character of first string, recursively
	# compute minimum cost for all three operations and take
	# minimum of three values.
	return 1 + min(editDistance(str1, str2, m, n-1), # Insert
				editDistance(str1, str2, m-1, n), # Remove
				editDistance(str1, str2, m-1, n-1) # Replace
				)

# Driver code
str1 = "เปลี่ยน"
str2 = "เปลี่ย"
print (editDistance(str1, str2, len(str1), len(str2)))

# This code is contributed by Bhavya Jain


1


# Corpus

In [ ]:
from pythainlp.spell import NorvigSpellChecker
from pythainlp.corpus import download , get_corpus_path , get_corpus

In [ ]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
len(checker.dictionary())
checker.dictionary()

dict_items([('ภูล', 6), ('กรูด', 25), ('ตับแข็ง', 39), ('ลี่ย์', 27), ('สังหาร', 693), ('แกรโนไดออไรต์', 6), ('อนินทรีย์', 37), ('ยูเนสโก', 91), ('ฉิ่ง', 71), ('เวลช์', 9), ('เล้ว', 28), ('จเร', 37), ('แร่', 648), ('นันทิ', 135), ('ทัณฑสถาน', 38), ('แจ้', 58), ('จิมส์', 2), ('คาสต์', 24), ('เสียเชิง', 13), ('เหียก', 4), ('จารย์', 414), ('ชูร์', 6), ('มัศ', 3), ('หัวเรื่อง', 81), ('ฉึ่ง', 18), ('แนร์', 13), ('คอนเฟอเรนซ์', 13), ('ร่วมประเวณี', 230), ('นิวพอร์ต', 13), ('ยิ้ว', 4), ('ไม้เป็น', 76), ('ตาเหลือก', 60), ('บาดตา', 65), ('ผลัม', 3), ('แด๊ฟ', 3), ('บังคลาเทศ', 51), ('มุ้งลวด', 33), ('หลอมละลาย', 61), ('เธส', 7), ('ด๊อย', 2), ('ซง', 86), ('จ้น', 9), ('อชิ', 155), ('ตัวเอก', 347), ('แป๊ะซะ', 6), ('ตลีย์', 7), ('ทุเรียน', 212), ('ครีโอล', 7), ('ขน', 4022), ('ป่าสูง', 21), ('ปล่อยมือ', 284), ('สกูป', 7), ('ก้านบัว', 14), ('ภูฏาน', 376), ('เกรียด', 2), ('พรหมวิหาร', 68), ('ตั้งแต่', 21505), ('ดาว', 8418), ('หลิ่น', 4), ('ส.ต.ท.', 17), ('สารบบ', 78), ('เซิร์ก', 8), ('อดอยาก', 200), ('

# Dictionary

In [ ]:
from pythainlp.corpus import download , get_corpus_path , get_corpus

In [ ]:
class unigram():

  def __init__(self,dict_path):    
    self.word = [item[0] for item in dict_path]
    self.freq = [item[1] for item in dict_path]
    self.df = pd.DataFrame(self.word,columns=["words"])
    self.df["freq"] = pd.DataFrame(self.freq)
    self.dictt = self.makedict()

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,word):
    return self.df.loc[word]
  
  def makedict(self):
    tmp_dict ={}
    for i in range(len(self.word)):
      tmp_dict[self.word[i]] = self.freq[i]

    return tmp_dict

  

In [ ]:
class Corpus():
  
  def __init__(self,dict_word_fre):
    self.dictt = dict_word_fre
    self.word = list(dict_word_fre.keys())
    self.fre = list(dict_word_fre.values())
    print("Complete init corpus")

  def __len__(self):
    return len(self.word)

  def __getitem__(self,word):
    return self.dictt[word]

class Corpus_from_dow(Corpus):
  def __init__(self,name):
    dict_cor = self.dow(name)
    Corpus.__init__(self,dict_cor)
  def dow(self,name):
    download(name)
    path = get_corpus_path(name)
    cor = get_corpus(path)
    word = ["".join(x.split("\t")[:-1]) for x in cor]
    fre = [int(x.split("\t")[-1]) for x in cor]
    dict_cor = dict(zip(word, fre))
    return dict_cor

In [ ]:
class candidater():
  def edits1(self,word):
    "All edits that are one edit away from `word`."    
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    # transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    #replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    # inserts    = [L + c + R               for L, R in splits for c in letters]   
    return set(deletes)#set(deletes + transposes + replaces + inserts)

  def delet2dict(self,dict_new,text):   
    for delet1 in self.edits1(text):
      if dict_new.get(delet1) is None:
        dict_new.update({
            delet1:{text}
            })
      else:
        dict_new.get(delet1).add(text)

  def __init__(self,unigrame):
    self.unidict = unigrame.dictt
    uniword = unigrame.word    
    dict_new = {}
    for text in uniword:     
      self.delet2dict(dict_new,text)
    self.dict_new = dict_new

  def candidate(self,word):
    set_candidate = set()

    if word in self.dict_new.keys():
      set_candidate.update(self.dict_new[word])
      
    del_set = self.edits1(word)

    for word_del in del_set:
      
      if word_del in self.unidict.keys():
        set_candidate.add(word_del)

      if word_del in self.dict_new.keys():
        set_candidate.update(self.dict_new[word_del])
    return set_candidate

In [ ]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
unigrame = Corpus(dict(checker.dictionary()))

Complete init corpus


In [ ]:
trigrame = Corpus_from_dow("tnc_trigram_word_freqs")
bigrame = Corpus_from_dow("tnc_bigram_word_freqs")

Corpus: tnc_trigram_word_freqs
- Downloading: tnc_trigram_word_freqs 2017


100%|██████████| 151914848/151914848 [00:09<00:00, 16386004.35it/s]


Complete init corpus
Corpus: tnc_bigram_word_freqs
- Downloading: tnc_bigram_word_freqs 2017


100%|██████████| 42200711/42200711 [00:00<00:00, 43443227.77it/s]


Complete init corpus


In [ ]:
candit = candidater(unigrame)

In [ ]:
%timeit candit.candidate('สงัดดี')

The slowest run took 4.96 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 7.01 µs per loop


In [ ]:
len(candit.candidate('แปล'))

93

# **Combine function to use for full senetnce**

In [ ]:
trigrame.dictt["<s/>แปลกแต่"]

'5'

In [ ]:
selected_words =[]
freq_words=[]

for item in tqdm(s):    
  try:
    freq_words = trigrame.dictt[item]
    print(freq_words)
    selected_words.append(item)
    freq_words.append(int(freq_words))
    print("freq_words")
  except:
    pass  

100%|██████████| 8/8 [00:00<00:00, 3830.41it/s]

5


In [ ]:
selected_words

['<s/>แปลกแต่']

In [ ]:
freq_words

'5'

In [ ]:
trigrame.dictt["<s/>แปลกแต่"]

'5'

In [ ]:
trigrame.dictt["<s/>และแต่"]

'5'

# Function area

In [ ]:
#combine function for find correct words
def correct_word_s(input_words):  
  #2. find possible word for each word
  word__wrong_list = input_words

  record_possible_word =[]
  
  
  for word in word__wrong_list:
    # each_word_list =[]
    each_word_list = candit.candidate(word)
    record_possible_word.append(each_word_list)  

  #3.combine each possible words to trigram word

  if len(record_possible_word) != 0:
    all_edit_words =[]
    for i in range(len(word__wrong_list)):
      # print(i)  
      for item in record_possible_word[i]:
        if i == 0:
          new_word = item + word__wrong_list[1] + word__wrong_list[2]
        elif i == 1:
          new_word = word__wrong_list[0] + item + word__wrong_list[2]
        else:
          new_word = word__wrong_list[0] + word__wrong_list[1] + item
        all_edit_words.append(new_word)
        
  else:
    print('no record possible word')

  #4. loop edit word in dict again to find most freq(prob)

  selected_words=[]
  freq_words=[]

  # print(all_edit_words)

  for item in all_edit_words:    
    try:
      freq_word = trigrame.dictt[item]
      selected_words.append(item)
      freq_words.append(int(freq_word))

    except:
      pass
  #after augment may be reduce to / words (bigrame dict)
  if len(selected_words) == 0:
    for item in all_edit_words:  
      try:
        freq_word = bigrame.dictt[item]
        selected_words.append(item)
        freq_words.append(int(freq_word))
       
      except:
        pass

  #after augment may be reduce to 1 words (unigrame dict)
  if len(selected_words) == 0:
    for item in all_edit_words:  
      if item in unigrame.word:
        selected_words.append(item)
        freq_words.append(unigrame[unigrame.df['words'] == item].freq)
    

  #select the most freq
  try:
    correct_word = selected_words[np.argmax(freq_words)]
    return correct_word
  except: #may be correct but wrong cut that why we cannot find it in dict   
    return "".join(input_words)

**Full sentence run**

In [ ]:
#combine all

def Pmick(sentence):

  def prepare_index(ans_list, length_acc_list):
    for idx, sen in enumerate(ans_list):
      if idx != 0:
        for each in sen:
          each['start']  += length_acc_list[idx-1]
          each['end']  += length_acc_list[idx-1]
          
    return ans_list

  ans_list = list()
  count_idx = 0

  # Find accumulative index in pg
  sentence_list = sent_tokenize(sentence)
  length_acc_list = list()
  length_acc = 0
  for i in sentence_list:
    length_acc += len(i)
    length_acc_list.append(length_acc)


  for text in sentence_list:
    word_deepcut =  word_tokenize(text, engine="deepcut") #deep or atta

    full_word=[]
    Wrong_word=[]
    #index
    start_idx_list=[]
    end_idx_list=[]

    #step1. 
    count = 1

    for idx in range(len(word_deepcut)):

      if idx == 0: #only first index to add </s> for first 2 words
        word=word_deepcut[idx:idx+2]

        start_idx_list.append(0) #add value 0
        count_word=''.join(word)    
        end_idx_list.append(len(count_word))   #add start+len  

        word.insert(0,'<s/>')
        full_word.append(word)

      if idx == len(word_deepcut)-2:
        word=word_deepcut[idx:idx+2]
        count_word=''.join(word)    
        
        start_idx_list.append(count-1)
        count+=len(count_word)
        end_idx_list.append(count-1)
        word.append('<s/>')
        full_word.append(word)
        break

      else:
        word=word_deepcut[idx:idx+3]
        count_word=''.join(word)    
        start_idx_list.append(count-1)    
        end_idx_list.append(count+len(count_word)-1)
        count+=len(word[0]) #next start index

      full_word.append(word)
      start_idx_list[1] = 0 #direct apply

    #step2.
    wrong_words =[]
    correct_words =[]
    wrong_word_combine=[]
    selected_start_index=[]
    selected_end_index=[]

    for i in range(len(full_word)):
      tri_word="".join(full_word[i])
      if tri_word in trigrame.word:
        pass   
      else:    
        wrong_words.append(full_word[i])
        wrong_word_combine.append(tri_word)
        selected_start_index.append(start_idx_list[i])
        selected_end_index.append(end_idx_list[i])


    edit_word_list=[]

    #find correct word
    for item in wrong_words:
      correct_one = correct_word_s(item)
      edit_word_list.append(correct_one)
      print(correct_one)

    #find dict list
    result =[]
    for i in range(len(edit_word_list)):
      edit_word ={}
      if wrong_word_combine[i] != edit_word_list[i]:    
        edit_word["start"] = selected_start_index[i]
        edit_word["end"] = selected_end_index[i]
        edit_word["old_word"] = wrong_word_combine[i]
        edit_word["new_word"] = edit_word_list[i]
        result.append(edit_word)

    ans_list.append(result)
  ans = prepare_index(ans_list, length_acc_list)
  return ans

In [ ]:
sentence = "แปลแต่จริงสิ่งนี้ไมน่าจะเกดขึ้นกันฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอปิดหัวใจให้รักเทอง่ายตาย วันนี้เป็นอย่างไจบ้าง ฉันอยากไหเที่ยว"
ans = Pmick(sentence)
ans

<s/>และแต่
แปลกแต่จริง
แต่จริงสิ่ง
จริงสิ่งนี้
สิ่งนี้ไม่
นี้ไม่น่า
ไม่น่าจะ
น่าจะเกิด
จะเกิดขึ้น
เกิดขึ้นกัน
ขึ้นกับฉัน
กันฉันบอก
ได้ยังไร
ยังไงเปลี่ยน
ไงเปลี่ยนคน
คนที่คอย
ที่รอปิด
คอปิดหัวใจ
เปิดหัวใจให้
หัวใจให้รักเทอ
ให้รักเทอง่าย
รักเทอง่ายตาย
ง่ายตาย 
ตาย <s/>
<s/>อันนี้
อันนี้เป็น
เป็นอย่างไจบ้าง
อย่างไจบ้าง 
ไจบ้าง <s/>
ฉันอยากไป
อยากไปเที่ยว
ไปเที่ยว<s/>


[[{'end': 6, 'new_word': '<s/>และแต่', 'old_word': '<s/>แปลแต่', 'start': 0},
  {'end': 10, 'new_word': 'แปลกแต่จริง', 'old_word': 'แปลแต่จริง', 'start': 0},
  {'end': 19, 'new_word': 'สิ่งนี้ไม่', 'old_word': 'สิ่งนี้ไม', 'start': 10},
  {'end': 22, 'new_word': 'นี้ไม่น่า', 'old_word': 'นี้ไมน่า', 'start': 14},
  {'end': 24, 'new_word': 'ไม่น่าจะ', 'old_word': 'ไมน่าจะ', 'start': 17},
  {'end': 27, 'new_word': 'น่าจะเกิด', 'old_word': 'น่าจะเกด', 'start': 19},
  {'end': 31, 'new_word': 'จะเกิดขึ้น', 'old_word': 'จะเกดขึ้น', 'start': 22},
  {'end': 34,
   'new_word': 'เกิดขึ้นกัน',
   'old_word': 'เกดขึ้นกัน',
   'start': 24},
  {'end': 37, 'new_word': 'ขึ้นกับฉัน', 'old_word': 'ขึ้นกันฉัน', 'start': 27},
  {'end': 62, 'new_word': 'ได้ยังไร', 'old_word': 'ได้ยังไง', 'start': 54},
  {'end': 76, 'new_word': 'คนที่คอย', 'old_word': 'คนที่คอ', 'start': 69},
  {'end': 79, 'new_word': 'ที่รอปิด', 'old_word': 'ที่คอปิด', 'start': 71},
  {'end': 87,
   'new_word': 'เปิดหัวใจให้',
   'old_word'

In [ ]:
def P(word,dict_word): 
  N=sum(dict_word.values())
  return dict_word[word] / N

def P_sort(word):
  if word in unigrame.dictt.keys():
    return P(word,unigrame.dictt)
  elif word in bigrame.dictt.keys():
    return P(word,bigrame.dictt)
  elif word in trigrame.dictt.keys():
    return P(word,trigrame.dictt)

In [ ]:
N = sum([int(x) for x in trigrame.dictt.values()])

def is_overlapped(a,b):
  if a[0] > b[0]: a,b = b,a
  if a[1] > b[0]: return True
  return False


def add_prop(sentence,candidates):
  sentence = list(sentence)
  window_size = 3
  result = []
  for candidate in candidates:
    start, end = candidate["start"], candidate["end"]
    sen = sentence[:]
    sen[start:end] = list(candidate["new_word"])
    segs = word_tokenize(''.join(sen), engine='attacut')
    prod = 1
    for i in range(len(segs) - window_size+1):
      gram = segs[i:i+window_size]
      gram = ''.join(gram)
      ele = trigrame[gram]
      if ele != None:
        prod += ele
    candidate.update({"prop":prod})
    result.append(candidate)
  return result
    

def remove_overlap(sentence, candidates):
  # candi = sorted(candidates_prop, key=lambda x: x['prop'])
  candi = add_prop(sentence, candidates)
  sen = sentence[:]
  i = 0
  list_candidate = set()
  while i < len(candi):
    a,b,c = None,None,None
    a = [candi[i]["start"], candi[i]["end"]]
    if i + 1 < len(candi):
      b = [candi[i+1]["start"], candi[i+1]["end"]]
    if i + 2 < len(candi):
      c = [candi[i+2]["start"], candi[i+2]["end"]]
    
    if b is None and c is None:
      list_candidate.add(candi[i]["new_word"])
      break
    
    if b is not None and c is None:
      if is_overlapped(a,b):
        if candi[i]['prop'] >= candi[i+1]['prop']:
          list_candidate.add(candi[i]["new_word"])
        else:
          list_candidate.add(candi[i+1]["new_word"])
      else:
        list_candidate.add(candi[i]["new_word"])
        list_candidate.add(candi[i+1]["new_word"])
      break

    else:
      if is_overlapped(a,b):
        if is_overlapped(a,c):
          agmax = np.argmax([candi[i]['prop'], candi[i+1]['prop'], candi[i+2]['prop']])
          list_candidate.add(candi[agmax]["new_word"])
          i += 3
        else:
          if candi[i]['prop'] >= candi[i+1]['prop']:
            list_candidate.add(candi[i]["new_word"])
          else:
            list_candidate.add(candi[i+1]["new_word"])
          i += 2
      else:
        list_candidate.add(candi[i]["new_word"])
        list_candidate.add(candi[i+1]["new_word"])
        list_candidate.add(candi[i+2]["new_word"])
        i += 3
  return list_candidate

In [ ]:
add_prop(sent_tokenize(sentence)[0], ans[0])

In [ ]:
remove_overlap(sent_tokenize(sentence)[0], ans[0])